In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [2]:
driver=webdriver.Chrome()

In [5]:
driver.get("https://www.aliexpress.com/w/wholesale-Living-Room-Sofas.html?spm=a2g0o.categorymp.0.0.37ed8c3S8c3S3M&categoryUrlParams=%7B%22q%22%3A%22Living%20Room%20Sofas%22%2C%22s%22%3A%22qp_nw%22%2C%22osf%22%3A%22category_navigate%22%2C%22sg_search_params%22%3A%22%22%2C%22guide_trace%22%3A%226b5d4458-c622-485a-8158-53dec020853d%22%2C%22scene_id%22%3A%2237749%22%2C%22searchBizScene%22%3A%22openSearch%22%2C%22recog_lang%22%3A%22en%22%2C%22bizScene%22%3A%22category_navigate%22%2C%22guideModule%22%3A%22category_navigate_vertical%22%2C%22postCatIds%22%3A%221503%2C15%22%2C%22scene%22%3A%22category_navigate%22%7D&isFromCategory=y")
# https://www.aliexpress.com/w/wholesale-Wok.html?isFromCategory=y&categoryUrlParams=%7B%22q%22%3A%22Wok%22%2C%22s%22%3A%22qp_nw%22%2C%22osf%22%3A%22category_navigate%22%2C%22sg_search_params%22%3A%22%22%2C%22guide_trace%22%3A%22c6caf7c3-5132-41c2-98d6-e2757a49fe47%22%2C%22scene_id%22%3A%2237749%22%2C%22searchBizScene%22%3A%22openSearch%22%2C%22recog_lang%22%3A%22en%22%2C%22bizScene%22%3A%22category_navigate%22%2C%22guideModule%22%3A%22category_navigate_vertical%22%2C%22postCatIds%22%3A%2215%2C13%22%2C%22scene%22%3A%22category_navigate%22%7D&page=2&g=y&SearchText=Wok
try:
    driver.maximize_window()
except:
    pass

In [4]:
wait=WebDriverWait(driver, 15)
wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="_full_container_header_23_"]/div[2]/div/div[2]/div[2]/div/div'))).click()
wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="_full_container_header_23_"]/div[2]/div/div[2]/div[2]/div[2]/div[6]/div/div[1]'))).click()
wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="_full_container_header_23_"]/div[2]/div/div[2]/div[2]/div[2]/div[6]/div/div[2]/div[3]'))).click()
wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="_full_container_header_23_"]/div[2]/div/div[2]/div[2]/div[2]/div[7]'))).click()

In [6]:
scroll_pause_time=2
scroll_step=300
last_height = driver.execute_script("return document.body.scrollHeight")
current_position = 0
while True:
            # Scroll down a small amount
    current_position = min(current_position + scroll_step, last_height)
    driver.execute_script(f"window.scrollTo(0, {current_position});")
                
                # Wait for new content to load
    time.sleep(scroll_pause_time)
                
                # Calculate new scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
                
                # Break if we've reached the bottom
    if current_position >= last_height:
                    # One final check for any dynamically loaded content
        time.sleep(scroll_pause_time * 2)
        final_height = driver.execute_script("return document.body.scrollHeight")
        if final_height == last_height:
            break
        last_height = final_height

In [7]:
products=driver.find_elements(by=By.CSS_SELECTOR,value='div.hm_hn div div div a')

In [8]:
len(products)

59

In [9]:
products_links=[product.get_attribute("href") for product in products if product.get_attribute("href").startswith("https://www.aliexpress.com/item")]

In [10]:
len(products_links)

59

In [12]:
data=[]

for prod_link in products_links:
    driver.get(prod_link)
    
    name=driver.find_element(by=By.CSS_SELECTOR,value='div.title--wrap--UUHae_g [data-pl="product-title"]').text

    price=driver.find_element(by=By.CSS_SELECTOR,value='[data-pl="product-price"] div span').text

    imgs=driver.find_elements(by=By.CSS_SELECTOR,value='div div.slider--item--RpyeewA div img')
    img_links=[img_link.get_attribute('src') for i,img_link in enumerate(imgs) if i<5]
    
    try:
        driver.find_element(by=By.XPATH,value='//*[@id="nav-specification"]/button').click()
    except:
        pass
    details=[]
    keys=driver.find_elements(by=By.CSS_SELECTOR,value='ul.specification--list--GZuXzRX li div div.specification--title--SfH3sA8')
    values=driver.find_elements(by=By.CSS_SELECTOR,value='ul.specification--list--GZuXzRX li div div.specification--desc--Dxx6W0W')

    for key,value in zip(keys,values):
        details.append(key.text)
        details.append(value.text)
        
    data.append({"Name":name,"Price":price,"Images":','.join(img_links),"Details":details})

    # for k in range(2,8):
    #     driver.find_element(by=By.XPATH,value='//*[@id="root"]/div[1]/div/div/div[2]/ul/li[10]/div/input').send_keys()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.title--wrap--UUHae_g [data-pl="product-title"]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A33CEFA5+77893]
	GetHandleVerifier [0x00007FF7A33CF000+77984]
	(No symbol) [0x00007FF7A31991BA]
	(No symbol) [0x00007FF7A31EF16D]
	(No symbol) [0x00007FF7A31EF41C]
	(No symbol) [0x00007FF7A3242237]
	(No symbol) [0x00007FF7A321716F]
	(No symbol) [0x00007FF7A323F07F]
	(No symbol) [0x00007FF7A3216F03]
	(No symbol) [0x00007FF7A31E0328]
	(No symbol) [0x00007FF7A31E1093]
	GetHandleVerifier [0x00007FF7A3687B6D+2931725]
	GetHandleVerifier [0x00007FF7A3682132+2908626]
	GetHandleVerifier [0x00007FF7A36A00F3+3031443]
	GetHandleVerifier [0x00007FF7A33E91EA+184970]
	GetHandleVerifier [0x00007FF7A33F086F+215311]


In [13]:
df=pd.DataFrame(data)
df.to_csv(f"couch.csv", index=False)

In [38]:
driver.get(products_links[0])

In [31]:
driver.find_element(by=By.CSS_SELECTOR,value='div.title--wrap--UUHae_g [data-pl="product-title"]').text

'Stainless Steel Wok Rack Pans Skillet Nonstick Cooking Pot Coating Honeycomb Everyday Traditional'

In [32]:
driver.find_element(by=By.CSS_SELECTOR,value='[data-pl="product-price"] div span').text

'PKR1,850'

In [36]:
imgs=driver.find_elements(by=By.CSS_SELECTOR,value='div div.slider--item--RpyeewA div img')

In [40]:
img_links=[img_link.get_attribute('src') for i,img_link in enumerate(imgs) if i<5]

In [41]:
img_links

['https://ae-pic-a1.aliexpress-media.com/kf/Sd66bc75286db4e3f9384b7063d12a7e3t.jpg_220x220q75.jpg_.avif',
 'https://ae-pic-a1.aliexpress-media.com/kf/S74c37f7b9f2040d5af9d24679392ffd4P.jpg_220x220q75.jpg_.avif',
 'https://ae-pic-a1.aliexpress-media.com/kf/S5fe270cc5d0f4e218a8099d0695066e93.jpg_220x220q75.jpg_.avif',
 'https://ae-pic-a1.aliexpress-media.com/kf/S9dfa8f3d74354b90ba79affc96baeec8m.jpg_220x220q75.jpg_.avif',
 'https://ae-pic-a1.aliexpress-media.com/kf/Sf849e903f49a4c188a19e43baebcd521k.jpg_220x220q75.jpg_.avif']

In [44]:
keys=driver.find_elements(by=By.CSS_SELECTOR,value='ul.specification--list--GZuXzRX li div div.specification--title--SfH3sA8')

In [45]:
len(keys)

13

In [46]:
values=driver.find_elements(by=By.CSS_SELECTOR,value='ul.specification--list--GZuXzRX li div div.specification--desc--Dxx6W0W')

In [47]:
len(values)

13

In [81]:
for i in range(1,9):
    page=driver.find_element(by=By.XPATH,value='//*[@id="root"]/div[1]/div/div/div[2]/ul/li[10]/div/input')
    # //*[@id="root"]/div[1]/div/div/div[2]/ul/li[11]/div/input
    # //*[@id="root"]/div[1]/div/div/div[2]/ul/li[12]/div/input
    page.send_keys(i)
    page.send_keys(Keys.ENTER)
    print(i)
    time.sleep(10)

1
2
3
4


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div[1]/div/div/div[2]/ul/li[10]/div/input"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71A42EFA5+77893]
	GetHandleVerifier [0x00007FF71A42F000+77984]
	(No symbol) [0x00007FF71A1F91BA]
	(No symbol) [0x00007FF71A24F16D]
	(No symbol) [0x00007FF71A24F41C]
	(No symbol) [0x00007FF71A2A2237]
	(No symbol) [0x00007FF71A27716F]
	(No symbol) [0x00007FF71A29F07F]
	(No symbol) [0x00007FF71A276F03]
	(No symbol) [0x00007FF71A240328]
	(No symbol) [0x00007FF71A241093]
	GetHandleVerifier [0x00007FF71A6E7B6D+2931725]
	GetHandleVerifier [0x00007FF71A6E2132+2908626]
	GetHandleVerifier [0x00007FF71A7000F3+3031443]
	GetHandleVerifier [0x00007FF71A4491EA+184970]
	GetHandleVerifier [0x00007FF71A45086F+215311]
